In [5]:
from src.dl_data_pipeline.validator import TypeValidator, ShapeValidator, MinMaxValidator, MeanVarValidator
from src.dl_data_pipeline.pipeline.data_pipeline import Pipeline
from tqdm import tqdm
import os
from src.dl_data_pipeline.process_functions import any_process
from src.dl_data_pipeline.process_functions import process_2d
import numpy as np

pipe = Pipeline()
pipe.add_forward_process(process_2d.open_rgb_image)
pipe.add_forward_process(any_process.rescale, min_value=0.0, max_value=1.0)
pipe.add_forward_process(process_2d.resize_with_max_distortion, (64, 64), 0.25, 0.0)
pipe.add_forward_process(process_2d.image_to_channel_num, 3, 0)
pipe.add_forward_process(lambda x : np.moveaxis(x, -1, 0))
pipe.add_forward_validator(TypeValidator(np.ndarray))
pipe.add_forward_validator(ShapeValidator((3, 64, 64)))
pipe.add_forward_validator(MinMaxValidator(0, 1))
# pipe.add_forward_validator(MeanVarValidator(0.5, 0.25, 0.1, 0.1))

pictures = []
for root, dirs, files in os.walk("../"):
    for file in files:
        if file.endswith(".png"):
            pictures.append(os.path.join(root, file))
    

Xdata = np.zeros((len(pictures), 3, 64, 64))
# for i, path in enumerate(tqdm(pictures)):
#     Xdata [i] = pipe.forward(path)
    # pipe.forward(np.random.rand(1080, 1920, 3))
print(pipe)

Pipeline(forward_processes=5, forward_validators=3, backward_processes=0, backward_validators=0)


In [22]:
import subprocess
import os

MIN_COVER = 75
log_path = os.path.normpath("test/test.log")
if os.path.exists(log_path):
    os.remove(log_path)
result = subprocess.run(
    ["python3", "-m", "pytest", "--cov=src", "--cov-report=html:test/htmlcov", f"--cov-fail-under={MIN_COVER}", "test/"],
    stdout=open(log_path, "w"),stderr=subprocess.STDOUT)
exc = result.returncode
if exc != 0:
    print(f"Test fails. Exit code {exc}")
    print(f"Check log file : {log_path} for more info")
else:
    if os.name == 'nt':
        os.startfile(os.path.normpath("test/htmlcov/index.html"))
    elif os.name == 'posix': 
        subprocess.run(["open", "test/htmlcov/index.html"])

Test fails. Exit code 1
Check log file : test/test.log for more info


In [19]:
from src.dl_data_pipeline import deferred
from src.dl_data_pipeline import Pipeline, DATA_PLACEHOLDER as DP
from src.dl_data_pipeline import process_functions
import numpy as np


# deferred.use_deferred_execution()

pipe = Pipeline()
pipe.add_forward_process(process_functions.any_process.rescale, 0, 1)

pipe.forward(np.random.rand(10)*50)

array([0.9695446 , 0.03420365, 1.        , 0.90440622, 0.00927965,
       0.78291949, 0.23193117, 0.89277142, 0.        , 0.58972763])